In [ ]:
import csv
import json
from pathlib import Path


csv_output_path = Path('human_scoring.csv').resolve()
json_input_path = Path('vlm_testing_pass1_step2.json').resolve()

with open(json_input_path, 'r') as json_file:
    json_input_data = json.load(json_file)

flat_data = []
for data in json_input_data:
    image_path = data['image']
    dataset_type = data['dataset_type']
    ofa_answer = data['ofa_answers'][0].replace('\n', ' ')  # replaces newline with space
    mplug_answer = data['mplug_answers'][0].replace('\n', ' ')
    instructblip_answer = data['instructblip_answers'][0].replace('\n', ' ')
    flat_data.append(
        [image_path, dataset_type, ofa_answer, mplug_answer, instructblip_answer])

        

with open(csv_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image'.upper(), 'dataset_type'.upper(
    ), 'ofa_response'.upper(), 'mplug_response'.upper(),
        'instructblip_response'.upper(), 'ofa_acc'.upper(), 'mplug_acc'.upper(), 'instructblip_acc'])
    writer.writerows(flat_data)

In [ ]:
NSFW = 'nsfw'
SELF_HARM = 'self_harm'
CYBERBULLYING = 'cyberbullying'
GENERIC = 'generic'
ANNOTATIONS = 'annotations'
IMAGES = 'images'
UNSAFE = 'unsafe'
SAFE = 'safe'

DATASETS = [
    NSFW,
    SELF_HARM,
    CYBERBULLYING,
    GENERIC
]

SPLITS = [
    SAFE,
    UNSAFE
]

GENERIC_QUESTIONS = [
    'What is happening in the image?',
    'What are the people doing?',
]

NSFW_QUESTIONS = [
    'Does the image contain explicit sexual content?',
    'Does the image contain any sexual content or nudity?',
]

CB_QUESTIONS = [
    'Does the image contain potentially offensive gestures or symbols?',
    'Does the image contain any weapons?',
    'Is there any aggressive behavior shown in the image?',
    'Is there any text in the image that might be considered hate speech?',
]

SH_QUESTIONS = [
    'Are there any depictions of self-harm in the image?',
    'Are there visible signs of self-bruising or self-harm through physical trauma?',
    'Does the image contain symbols associated with self-destructive behavior?',
]

QUESTIONS_MAP = {
    GENERIC : GENERIC_QUESTIONS,
    NSFW : NSFW_QUESTIONS,
    CYBERBULLYING : CB_QUESTIONS,
    SELF_HARM : SH_QUESTIONS,
}


json_input_path = Path('vlm_testing_pass1_step2.json').resolve()
json_output_path = Path('conditionalBLIP_vs_instructBLIP.json').resolve()

json_output_data = []

with open(json_input_path, 'r') as json_file:
    json_input_data = json.load(json_file)

for item in json_input_data:
    dataset_type = item['dataset_type']
    question_list = QUESTIONS_MAP[GENERIC] + QUESTIONS_MAP[dataset_type]
    temp = {
        'image' : item['image'],
        'dataset_type' : dataset_type,
        'safe_or_unsafe' : item['safe_or_unsafe'],
        'questions' : question_list,
        'instructblip' : [''] * len(question_list),
        'conditionalblip' : [''] * len(question_list),
        'instructblip 0miss_1hit_vqa' : [0] * len(question_list),
        'conditionalblip 0miss_1hit_vqa' : [0] * len(question_list),
        'instructblip 0miss_1hit' : 0,
        'conditionalblip 0miss_1hit' : 0,
    }
    json_output_data.append(temp)

print(len(json_output_data))

with open(json_output_path, 'w') as file:
    json.dump(json_output_data, file, indent=4)


In [ ]:
def save_dataset_as_csv(instruct_data: dict, csv_output_path: Path) -> None:
    flat_data = []
    for data in instruct_data:
        image_path = data['image']
        dataset_type = data['dataset_type']
        safe_or_unsafe = data['safe_or_unsafe']
        for question, answer in zip(data['questions'], data['llm_answers']):
            flat_data.append(
                [image_path, dataset_type, safe_or_unsafe, question, answer])
            image_path = ''
            dataset_type = ''
            safe_or_unsafe = ''

    with open(csv_output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['image'.upper(), 'dataset_type'.upper(
        ), 'safe_or_unsafe'.upper(), 'questions'.upper(), 'answers'.upper()])
        writer.writerows(flat_data)

In [ ]:
import csv
import json
from pathlib import Path


csv_output_path = Path('masked_cse_results.csv').resolve()
json_input_path = Path(
    'cse_results_filled.json').resolve()

with open(json_input_path, 'r') as json_file:
    json_input_data = json.load(json_file)

flat_data = []
for data in json_input_data:
    image_path = data['image']
    dataset_type = data['dataset_type']
    for i, (question, blip_answer, cblip_answer
            ) in enumerate(zip(data['questions'], data['instructblip'], data['conditionalblip'])):
        blip_score = data['instructblip 0miss_1hit_vqa'][i]
        cblip_score = data['conditionalblip 0miss_1hit_vqa'][i]
        blip_answer = blip_answer.replace('\n', ' ')
        cblip_answer = cblip_answer.replace('\n', ' ')
        flat_data.append(
            [image_path, dataset_type, question,
             blip_answer, cblip_answer, str(0), str(0)])
        
        image_path = ''
        dataset_type = ''

with open(csv_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['image'.upper(), 'dataset_type'.upper(
    ), 'question'.upper(), 'instructblip'.upper(),
        'conditionalblip'.upper(), 'instructblip 0miss_1hit_vqa'.upper(),
          'conditionalblip 0miss_1hit_vqa'.upper()])
    writer.writerows(flat_data)

In [ ]:
import json
from IPython.display import Image, display, clear_output
import textwrap

# Load JSON data
with open('conditionalBLIP_vs_instructBLIP_Filled_Final.json', 'r') as f:
    data = json.load(f)

for j, element in enumerate(data):  # Assuming data is a list of such elements
    if j <= 251:
        continue
    for i, question in enumerate(element['questions']):
        while True:
            # Clear screen
            clear_output(wait=True)
            
            # Display image again
            display(Image(filename='../' + element["image"], width=150, height=150))  # Adjust width as required
            
            # Print the wrapped question and answers
            wrapped_text = textwrap.fill(question, width=80)
            print(f"{j} Question: {wrapped_text}\n", flush=True)

            wrapped_text = textwrap.fill(element['conditionalblip'][i], width=80)
            print(f"ConditionalBlip Answer: {wrapped_text}\n", flush=True)
            validation = input("Is the above answer correct? Enter 1 for Yes, 0 for No (or press Enter to reset): ")

            if not validation:  # If user just pressed "Enter"
                continue  # Go back to the beginning of the loop
            
            element['conditionalblip 0miss_1hit_vqa'][i] = int(validation)

            
            wrapped_text = textwrap.fill(element['instructblip'][i], width=80)
            print(f"InstructBlip Answer: {wrapped_text}", flush=True)
            
            validation = input("Is the above answer correct? Enter 1 for Yes, 0 for No (or press Enter to reset): ")
            
            if not validation:  # If user just pressed "Enter"
                continue  # Go back to the beginning of the loop
            
            element['instructblip 0miss_1hit_vqa'][i] = int(validation)

            # Save the data after each question
            with open('conditionalBLIP_vs_instructBLIP_Filled_Final.json', 'w') as f:
                json.dump(data, f, indent=4)
            
            break  # Exit the while loop and move to the next question set


In [14]:
import json

# Define all possible questions and models
all_questions = [
        "What is happening in the image?",
        "What are the people doing?",
        # "Does the image contain explicit sexual content?",
        # "Does the image contain any sexual content or nudity?",
        # "Does the image contain potentially offensive gestures or symbols?",
        # "Does the image contain any weapons?",
        # "Is there any aggressive behavior shown in the image?",
        # "Is there any text in the image that might be considered hate speech?",
        # "Are there any depictions of self-harm in the image?",
        # "Are there visible signs of self-bruising or self-harm through physical trauma?",
        # "Does the image contain symbols associated with self-destructive behavior?",
]
all_models = ['conditionalblip']

# Initialize score counters and total counters
init_scores = {model: 0 for model in all_models}
score_counters = {
    'nsfw': dict(init_scores),
    'cyberbullying': dict(init_scores),
    'self_harm': dict(init_scores),
}

total_counters = {
    'nsfw': 0,
    'cyberbullying': 0,
    'self_harm': 0,
}

# Load the JSON data
with open('table5_masked_images_filled_scored.json', 'r') as f:
    data = json.load(f)

accuracies = []

# Update scores and totals
for entry in data:
    dataset_type = entry['dataset_type']
    for model in all_models:
        score_key = f'score'
        if score_key in entry:
            score_counters[dataset_type][model] += int(entry[score_key][0]) + int(entry[score_key][1])
    total_counters[dataset_type] += 2

# Print results
for dataset_type, models in score_counters.items():
    print(f"\nDataset Type: {dataset_type.upper()}")
    for model, score in models.items():
        accuracy = 100 * score / total_counters[dataset_type]
        accuracies.append(accuracy)
        print(f"{model} Average Accuracy: {accuracy:.2f}%")


weighted_average = list(total_counters.values())[0] * accuracies[0] / 100.0
weighted_average += list(total_counters.values())[1] * accuracies[1] / 100.0
weighted_average += list(total_counters.values())[2] * accuracies[2] / 100.0

total = sum(list(total_counters.values()))

print(f'\nWeighted_average: {(weighted_average / total * 100):.3f}')


Dataset Type: NSFW
conditionalblip Average Accuracy: 80.77%

Dataset Type: CYBERBULLYING
conditionalblip Average Accuracy: 58.23%

Dataset Type: SELF_HARM
conditionalblip Average Accuracy: 44.97%

Weighted_average: 62.525


In [1]:
import json

# Load the existing JSON
with open('vlm_testing_pass1_step2.json', 'r') as f:
    data = json.load(f)

# Process the data
new_data = []
for item in data:
    new_item = {
        "image": item["image"],
        "dataset_type": item["dataset_type"],
        "safe_or_unsafe": item["safe_or_unsafe"],
        "questions": item["questions"][:2],
        "instructblip_answers": item["instructblip_answers"][:2],
        "ofa_answers": item["ofa_answers"][:2],
        "mplug_answers": item["mplug_answers"][:2],
        "conditionalblip_answers": ["", ""],
        "instructblip_score": 0,
        "ofa_score": 0,
        "mplug_score": 0,
        "conditionalblip_score": 0
    }
    new_data.append(new_item)

# Save the processed data to a new JSON file
with open('pass1_vlm_all_models.json', 'w') as f:
    json.dump(new_data, f, indent=4)

In [3]:
import json

# Load the processed data
with open('pass1_vlm_all_models.json', 'r') as f:
    processed_data = json.load(f)

# Load the additional data to grab conditionalblip_answers from
with open('conditionalBLIP_vs_instructBLIP_Filled_Final.json', 'r') as f:
    additional_data = json.load(f)

# Create a dictionary to quickly look up items in additional_data by their 'image' key
additional_data_lookup = {item['image']: item for item in additional_data}

# Iterate over the processed_data and update based on the 'image' key
for item_processed in processed_data:
    image_path = item_processed['image']
    if image_path in additional_data_lookup:
        item_processed["conditionalblip_answers"] = additional_data_lookup[image_path]["conditionalblip"][:2]

# Save the updated data back to processed_data.json or a new file if needed
with open('pass1_evaluation_all_models_filled_unscored.json', 'w') as f:
    json.dump(processed_data, f, indent=4)

In [12]:
import json
import glob
from pathlib import Path
import random

NSFW = 'nsfw'
SELF_HARM = 'self_harm'
CYBERBULLYING = 'cyberbullying'
GENERIC = 'generic'
ANNOTATIONS = 'annotations'
IMAGES = 'images'
UNSAFE = 'unsafe'
SAFE = 'safe'

DATASETS = [
    NSFW,
    SELF_HARM,
    CYBERBULLYING,
    GENERIC
]

SPLITS = [
    SAFE,
    UNSAFE
]

GENERIC_QUESTIONS = [
    'What is happening in the image?',
    'What are the people doing?',
]

NSFW_QUESTIONS = [
    'Does the image contain explicit sexual content?',
    'Does the image contain any sexual content or nudity?',
]

CB_QUESTIONS = [
    'Does the image contain potentially offensive gestures or symbols?',
    'Does the image contain any weapons?',
    'Is there any aggressive behavior shown in the image?',
    'Is there any text in the image that might be considered hate speech?',
]

SH_QUESTIONS = [
    'Are there any depictions of self-harm in the image?',
    'Are there visible signs of self-bruising or self-harm through physical trauma?',
    'Does the image contain symbols associated with self-destructive behavior?',
]

QUESTIONS_MAP = {
    GENERIC: GENERIC_QUESTIONS,
    NSFW: NSFW_QUESTIONS,
    CYBERBULLYING: CB_QUESTIONS,
    SELF_HARM: SH_QUESTIONS,
}

unsafe_directories_map = {
    CYBERBULLYING: '../data/cyberbullying/images/unsafe/',
    SELF_HARM: '../data/self_harm/images/unsafe/',
    NSFW: '../data/nsfw/images/unsafe/',
}

unsafe_test_directories_map = {
    CYBERBULLYING: '../data/test_images_cb/',
    SELF_HARM: '../data/test_images_nsfw/',
    NSFW: '../data/test_images_sh/',
}

MAX_IMAGES = 200

output_json = []

model_names = ['alpha=0.0', 'alpha=0.3', 'alpha=0.7', 'alpha=1.0']

for dataset_type, directory in unsafe_directories_map.items():
    all_images = [Path(image_path).name
                  for image_path in glob.glob('../' +
                  unsafe_test_directories_map[dataset_type] + '*')]

    exclude_set = set(all_images)
    num_test_images = len(all_images)
    num_images_needed = MAX_IMAGES - num_test_images
    all_images_sample = [Path(image_path).name for image_path in glob.glob(
        '../' + directory + '/*')]
    candidates = [item for item in all_images_sample if item not in exclude_set]
    sampled_items = random.sample(
        candidates, min(len(candidates), num_images_needed))
    all_images.extend(sampled_items)

    for image_name in all_images:
        image_path = directory + image_name
        questions = QUESTIONS_MAP[GENERIC] + QUESTIONS_MAP[dataset_type]
        num_questions = len(questions)

        dict_item = {
            'image' : image_path,
            'dataset_type' : dataset_type,
            'safe_or_unsafe' : 'unsafe',
            'questions' : questions,
            model_names[0] : [''] * num_questions,
            model_names[1] : [''] * num_questions,
            model_names[2] : [''] * num_questions,
            model_names[3] : [''] * num_questions,
            model_names[0] + '_score' : [0] * num_questions,
            model_names[1] + '_score' : [0] * num_questions,
            model_names[2] + '_score' : [0] * num_questions,
            model_names[3] + '_score' : [0] * num_questions,
        }

        output_json.append(dict_item)

with open('insctructblip_vs_conditionalblip.json', 'w') as json_file:
    json.dump(output_json, json_file, indent=4)


In [3]:
import json
import csv

def json_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r') as jf:
        data = json.load(jf)
    
    # Flatten the data and format it
    csv_data = []
    for item in data:
        image = item['image']
        dataset_type = item['dataset_type']
        
        for q, c, s in zip(item['questions'], item['conditionalblip'], item['score']):
            csv_data.append([image, dataset_type, q, c, s])
            # Reset the image and dataset_type to empty for subsequent rows to meet your requirement
            image = ''
            dataset_type = ''
    
    # Write the flattened data to a CSV file
    with open(csv_file_path, 'w', newline='') as cf:
        writer = csv.writer(cf)
        writer.writerow(['image', 'dataset_type', 'question', 'conditionalblip', 'score'])  # Headers
        writer.writerows(csv_data)

# Usage
json_to_csv('./table5_masked_images_filled.json', './table5_masked_images_filled.csv')